<a href="https://colab.research.google.com/github/sugihara-16/CNN_from_scratch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [ ]:
!pip install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
!jupyter contrib nbextension install --user
!jupyter nbextension enable hinterland/hinterland

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Robot Intelligence


# MNISTの読み込み読み込み(tensorflow)

In [27]:
from tensorflow.keras.datasets import mnist 
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path='mnist.npz')

In [29]:
!ps aux
!kill -9 -1 <pid>

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0    992     4 ?        Ss   02:11   0:00 /sbin/docker-
root           7  0.1  0.4 349800 60220 ?        Sl   02:11   0:15 /tools/node/b
root          17  0.0  0.0  35888  4768 ?        Ss   02:11   0:01 tail -n +0 -F
root          30  0.0  0.0      0     0 ?        Z    02:11   0:06 [python3] <de
root          31  0.0  0.3 160352 41864 ?        S    02:11   0:00 python3 /usr/
root          44  0.2  0.4 193896 60588 ?        Sl   02:11   0:22 /usr/bin/pyth
root          45  0.1  0.0 708456 11048 ?        Sl   02:11   0:08 /usr/local/bi
root          60 35.0  5.2 3021396 700320 ?      Ssl  02:17  45:48 /usr/bin/pyth
root          80  0.2  0.1 128664 16464 ?        Sl   02:17   0:18 /usr/bin/pyth
root         370  0.0  0.0  59040  6396 ?        R    04:27   0:00 ps aux
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `kill -9 -1 <pid>'


# CNNレイヤー

In [5]:

from numpy.testing._private.utils import break_cycles
import random
import numpy as np

def MakeNet(input_size, filter_num, filter_size, affine_size,output_size):
  # heの初期値
  f = np.random.randn(filter_size**2, filter_num)/np.sqrt(filter_size**2) * np.sqrt(2)
  bf = np.zeros((1,10))
  w_a1 = np.random.randn(input_size, affine_size)/np.sqrt(input_size) * np.sqrt(2)
  b1 = 0
  w_a2 = np.random.randn(affine_size, output_size)/np.sqrt(affine_size) * np.sqrt(2)
  b2 = 0
  w_o = np.random.randn(output_size, output_size) /np.sqrt(output_size) * np.sqrt(2)
  return f, bf, w_a1, b1, w_a2, b2, w_o

def paddig(item):
  item_pad = np.pad(item, ((2, 2),))
  return item_pad

def InputNode(item_pad_std):
  input = []
  for i in range(28):
    for j in range(28):
      target = item_pad_std[i:i+5,j:j+5]
      target_reshape = np.array(target).reshape(1,25)[0]
      input.append(target_reshape)
  input = np.array(input).reshape(28*28,-1)
  return input

def zscore(x):
  xmean = np.mean(x)
  xstd  = np.std(x)
  zscore = (x-xmean)/xstd
  input = np.array(zscore).reshape(32,-1)
  return input

def Convolution(input,f,bf):
  bf_tile = np.tile(bf, (784,1))
  u_c = np.ravel((np.dot(input,f) + bf_tile).T)
  X_conv = np.maximum(u_c, 0) #　出力を一元化
  return X_conv, u_c

def Pooling(X_conv):
  X_p = []
  for i in range(int(7840/16)):
    target = X_conv[i*16:(i+1)*16]
    max_target = np.max(target) 
    arg_max = np.argmax(target)
    target[arg_max] += 1.0
    target_pooling = np.heaviside(target-max_target - 1.0, max_target)
    X_p.append(target_pooling)
  X_p = np.ravel(np.array(X_p))
  return X_p

def Affine(input,w_a,b):
  u = np.dot(input, w_a) + b
  u = np.array(u, dtype=float)
  #ReLu関数で活性化
  X = np.ravel(np.maximum(u, 0))
  return X, u

def SoftMax(x):
  x = np.array(x, dtype= float)
  x_max = max(x)
  exp_x = np.exp(x-x_max)
  sum_exp_x = np.sum(exp_x)

  y = exp_x/sum_exp_x #出力(入力画像がが0~9である確率)
  return np.array([y])

def OutPut(X3,w_o):
  x = np.dot(X3,w_o)
  p = SoftMax(x)
  return p

def Answer(label):
  sample = [0,1,2,3,4,5,6,7,8,9]
  t = np.zeros(10)
  for i , item in enumerate(sample):
    if int(item) == int(label):
      t[i] = 1
      break
  return t

def CalcDelta(p,t,u_c, u_a1, u_a2, X_p, w_o, w_a2, w_a1):
  if np.argmax(p) == np.argmax(t):
    judge = True
  else:
    judge = False
  d_o = p - t
  d_a2 = np.dot(d_o, w_o.T)*np.heaviside(u_a2,0) #ReLu関数の微分であるヘヴィサイド関数をかける
  d_a1 = np.dot(d_a2, w_a2.T)*np.heaviside(u_a1,0)
  d_p = np.dot(d_a1, w_a1.T)
  d_c = (d_p*np.heaviside(u_c, 0)).reshape(10,-1)
  return d_o, d_a1, d_a2, d_p, d_c, judge

def Update(input, X_p, X_a1, X_a2, f, bf, w_a1, b1, w_a2, b2, w_o, d_o, d_a1, d_a2, d_c, e):
  f = f - e*(np.dot(d_c, input)).T
  #f = f
  for i in range(len(bf)):
    target = d_c[i]
    target_sum = np.sum(target)
    bf[i] = bf[i] - e*target_sum
  #bf = bf
  w_a1 = w_a1 - e*np.dot(np.array([X_p]).T,d_a1)
  b1 = b1 - e*d_a1
  w_a2 = w_a2 - e*np.dot(np.array([X_a1]).T,d_a2)
  b2 = b2 - e*d_a2
  w_o = w_o - e*np.dot(np.array([X_a2]).T, d_o)
  return f, bf, w_a1, b1, w_a2, b2, w_o

# 実行

###ネットワークを学習

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 学習後の各パラメータ保存用の変数
f_learned, bf_learned, w_a1_learned, b1_learned, w_a2_learned, b2_learned, w_o_learned = MakeNet(28*28*10, #プーリング層のユニット数
                      10,#フィルター数
                      5, #フィルターサイズ
                      350, #全結合のユニット数
                      10) #出力層のユニット数
#初期フィルター
f_first = np.ones_like((25,10))

def AddNoize(source, rate):
  for i , row in enumerate(source):
    for j, item in enumerate(row):
      p = np.random.randint(100)
      if p < rate:
        source[i][j] = np.random.randint(256)
  return source

def main():
  global f_first, f_learned, bf_learned, w_a1_learned, b1_learned, w_a2_learned, b2_learned, w_o_learned 
  
  """""""""""""""""""""""""""
  ---ニューラルネットを定義---
  """""""""""""""""""""""""""

  f, bf, w_a1, b1, w_a2, b2, w_o = MakeNet(28*28*10, #プーリング層のユニット数
                        10,#フィルター数
                        5, #フィルターサイズ
                        350, #全結合のユニット数
                        10) #出力層のユニット数
  f_first = f       
  """""""""""""""""""""""""""
  ---ニューラルネットの学習---
  """""""""""""""""""""""""""
  count = 0
  for i , item in enumerate(train_images):
    item = train_images[i]
    item_noised = AddNoize(item, 0) #ノイズ追加
    label = train_labels[i]
    e = 0.001
    item_pad = paddig(item_noised) #padding
    item_pad_std = zscore(np.array(item_pad).reshape(1,32*32)[0]) #標準化
    input = InputNode(item_pad_std) #入力層用にデータ成形
    X_conv,u_c = Convolution(input,f,bf) #畳み込み層の出力を計算
    X_p = Pooling(X_conv) #プーリング層の出力を計算

    X_a1,u_a1 = Affine(X_p, w_a1, b1) #全結合層の出力を計算
    X_a2,u_a2 = Affine(X_a1, w_a2, b2) #全結合層の出力を計算
    p = OutPut(X_a2, w_o) #事後確率を計算
    t = Answer(label)
    d_o, d_a1, d_a2, d_p, d_c, judge = CalcDelta(p,t,u_c, u_a1, u_a2, X_p, w_o, w_a2, w_a1)
    f, bf, w_a1, b1, w_a2, b2, w_o = Update(input, X_p, X_a1, X_a2, f, bf, w_a1, b1, w_a2, b2, w_o, d_o, d_a1, d_a2, d_c, e)
    print(i)

  #学習結果を記録
  f_learned, bf_learned, w_a1_learned, b1_learned, w_a2_learned, b2_learned, w_o_learned = f, bf, w_a1, b1, w_a2, b2, w_o
main()

### ネットワークをテスト

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist 
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path='mnist.npz')

def AddNoize(source, rate):
  for i , row in enumerate(source):
    for j, item in enumerate(row):
      p = np.random.randint(100)
      if p < rate:
        source[i][j] = np.random.randint(256)
  return source

def main():
  f, bf, w_a1, b1, w_a2, b2, w_o = f_learned, bf_learned, w_a1_learned, b1_learned, w_a2_learned, b2_learned, w_o_learned 
  """""""""""""""""""""""""""
  ---ニューラルネットのテスト---
  """""""""""""""""""""""""""
  accuracy_sum = 0
  for j in range(10):
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data(path='mnist.npz')
    count = 0
    for i , item in enumerate(test_images[:100]):
      item = test_images[i]
      item_noised = AddNoize(item, 0) #ノイズを追加(第二引数にノイズ率を入力入力s)
      label = test_labels[i]
      e = 0.001
      item_pad = paddig(item_noised) #padding
      item_pad_std = zscore(np.array(item_pad).reshape(1,32*32)[0]) #標準化
      input = InputNode(item_pad_std) #入力層用にデータ成形
      X_conv,u_c = Convolution(input,f,bf) #畳み込み層の出力を計算
      X_p = Pooling(X_conv) #プーリング層の出力を計算
      X_a1,u_a1 = Affine(X_p, w_a1, b1) #全結合層の出力を計算
      X_a2,u_a2 = Affine(X_a1, w_a2, b2) #全結合層の出力を計算
      p = OutPut(X_a2, w_o) #事後確率を計算
      t = Answer(label)
      d_o, d_a1, d_a2, d_p, d_c, judge = CalcDelta(p,t,u_c, u_a1, u_a2, X_p, w_o, w_a2, w_a1)
      if judge:
        count += 1
    accuracy_sum += count/100
  print(accuracy_sum/10)
main()

### 可視化

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)


for i in range(10):
  f1 = (f_first.T[i]).reshape(5,-1)
  ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
  ax.imshow(f1, cmap=plt.cm.gray_r, interpolation='nearest')

plt.show()
plt.savefig('filters1.png')

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(10):
  f1 = (f_learned.T[i]).reshape(5,-1) 
  ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
  ax.imshow(f1, cmap=plt.cm.gray_r, interpolation='nearest') 
plt.show()